In [1]:
from __future__ import print_function
import os

import pandas as pd
from PIL import Image
import numpy as np

import chainer
from chainer.dataset import convert
import chainer.links as L
import chainer.functions as F
from chainer import serializers

from chainer.datasets import get_cifar10
from chainer.datasets import get_cifar100

import utils

Using TensorFlow backend.
/home/fytroo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


##### tensorboard用

In [2]:
from tb_chainer import SummaryWriter, name_scope, within_name_scope
try:
    writer = SummaryWriter('runs/{}_{}'.format(
        __file__, utils.now(isabout=True)))
except:
    writer = SummaryWriter('runs/'+utils.now(isabout=True))

### ハイパーパラメータ

In [3]:
from easydict import EasyDict
args = EasyDict({
    'bs': 64, 
    'epoch' : 100,
    'lr' : 0.05,
    'gpu': 0,
    'out': 'result',
    'resume': '',
    'n_in': 32, 
})
try:
    __file__.endswith('py')
    import argparse
    parser = argparse.ArgumentParser(description='Chainer example: MNIST')
    parser.add_argument('--batchsize', '-b', dest='bs', type=int, default=args.bs,
                        help='Number of images in each mini-batch')
    parser.add_argument('--epoch', '-e', type=int, default=args.epoch,
                        help='Number of sweeps over the dataset to train')
    parser.add_argument('--learningrate', '-l', dest='lr', type=float, default=args.lr,
                        help='Number of sweeps over the dataset to train')
    parser.add_argument('--frequency', '-f', type=int, default=-1,
                        help='Frequency of taking a snapshot')
    parser.add_argument('--gpu', '-g', type=int, default=args.gpu,
                        help='GPU ID (negative value indicates CPU)')
    parser.add_argument('--out', '-o', default=args.out,
                        help='Directory to output the result')
    parser.add_argument('--resume', '-r', default=args.resume,
                        help='Resume the training from snapshot')
    parser.add_argument('--unit', '-u', dest='n_in', type=int, default=args.n_in,
                        help='Number of units')
    parser.add_argument('--noplot', dest='plot', action='store_false',
                        help='Disable PlotReport extension')
    args = parser.parse_args()
except:
    print('no argsparse')
    pass

no argsparse


### データセット読み込み

In [4]:
dataset_dir = 'flower_images'
df  = pd.DataFrame.from_csv(os.path.join(dataset_dir,'flower_labels.csv'))

df['name'] = df.index
df['path'] = dataset_dir + '/' + df['name']

n_label = df.label.drop_duplicates().count()
n_label
df

/home/fytroo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


,label,name,path
file,,,
0001.png,0,0001.png,flower_images/0001.png
0002.png,0,0002.png,flower_images/0002.png
0003.png,2,0003.png,flower_images/0003.png
0004.png,0,0004.png,flower_images/0004.png
0005.png,0,0005.png,flower_images/0005.png
0006.png,1,0006.png,flower_images/0006.png
0007.png,6,0007.png,flower_images/0007.png
0008.png,0,0008.png,flower_images/0008.png
0009.png,0,0009.png,flower_images/0009.png


In [5]:
def load_fromdf(dataframe, resize=(96,96)):
    if type(resize) is int:
        resize = (resize, resize)
    
    df = dataframe
    x_data = []
    y_data = []
    for idx, row in df.iterrows():
        y = row['label']
        f = row['path']

        img = Image.open(f).resize(resize, Image.LANCZOS)
        img = img.convert('RGB')
        x = np.array(img)
        x_data.append(x)
        y_data.append(y)
    x_data = np.array(x_data)
    y_data = np.array(y_data)

    return x_data, y_data

In [6]:
df_train, df_test = utils.train_test_split_df(df, test_size=0.1)
x_train, y_train = load_fromdf(df_train, resize=args.n_in)
x_test, y_test = load_fromdf(df_test, resize=args.n_in)

x_train.shape

(200, 32, 32, 3)

### モデルを定義

In [7]:
class Fire(chainer.Chain):
    def __init__(self, n_in=None, n_out=32):
        super(Fire, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(n_in, 32, 3)
            self.conv2 = L.Convolution2D(None, n_out, 3)
            self.bn = L.BatchNormalization(32)
            self.bn2 = L.BatchNormalization(32)
            self.bn3 = L.BatchNormalization(32)
    
    def __call__(self, x):
        x = self.bn(x)
        x = self.conv1(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.bn3(x)
        return x    

In [8]:
class LeNet(chainer.Chain):
    def __init__(self, n_in=32, n_out=10):
        super(LeNet, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(None, 32, 3)
            self.fire1 = Fire(None, 32)
            self.fire2 = Fire(None, 32)
            self.fc = L.Linear(None, n_out)
    
    
    @within_name_scope('LeNet')
    def __call__(self, x):
        with name_scope('top_conv', self.conv1.params()):
            x = self.conv1(x)
            #x = F.dropout(x, 0.25)
        with name_scope('fire1', self.fire1.params()):
            x = self.fire1(x)
            #x = F.dropout(x, 0.25)
        with name_scope('fire2', self.fire2.params()):
            x = self.fire2(x)
            #x = F.dropout(x, 0.25)
        with name_scope('fc', self.fc.params()):
            x = F.average_pooling_2d(x, ksize=2, stride=2)
            x = self.fc(x)
        return x    

In [9]:
model = L.Classifier(LeNet(args.n_in, n_label),
                    lossfun=F.softmax_cross_entropy,
                    accfun=F.accuracy)
xp = np
if args.gpu >= 0:
    import cupy as cp
    xp = cp
    chainer.cuda.get_device_from_id(args.gpu).use()
    model.to_gpu()  # Copy the model to the GPU
optimizer = chainer.optimizers.MomentumSGD(args.lr)
optimizer.setup(model)

### data augmentation

In [10]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

import Augmentor
p = Augmentor.Pipeline()
p.flip_left_right(probability=0.5)
p.flip_top_bottom(probability=0.5)
g = p.keras_generator_from_array(x_train, y_train, batch_size=args.bs)
g = ((
    xp.array(np.swapaxes((x/255.), 1, 3)).astype(np.float32),
    xp.array(y.astype(np.int8))
    ) for (x,y) in g)

chainer.trainingを使わず，訓練ループをかく
chainer.trainingでは，自前のデータのイテレータを使うことができないため．
Augmentorを使いたい

### 訓練と検証

In [11]:
def train(step=None):
    total_loss = 0
    total_acc = 0
    n_data = 0
    n_train = len(y_train)
    for _ in range(n_train//args.bs):
        xs, ts = next(g) 
        x = chainer.Variable(xs)
        t = chainer.Variable(ts)
        optimizer.update(model, x, t)
        with chainer.using_config('train', True):
            loss = model(x,t)
        n_data += len(t.data)
        total_loss += float(loss.data) * len(t.data)
        total_acc += float(model.accuracy.data) * len(t.data)

    loss = total_loss / n_data
    acc = total_acc / n_data
    print('loss: {:.4f}\t acc: {:.4f}'.format(loss, acc))
    writer.add_scalar('loss', loss, step)
    writer.add_scalar('accuracy', acc, step)

def test(step=None):
    xs = xp.array(np.swapaxes((x_test), 1, 3)).astype(np.float32)
    ts = xp.array(y_test).astype(np.int8)
    x = chainer.Variable(xs)
    t = chainer.Variable(ts)
    loss = model(x,t)

    n_data = len(t.data)
    total_loss = float(loss.data) * len(t.data)
    total_acc = float(model.accuracy.data) * len(t.data)
    loss = total_loss / n_data
    acc = total_acc / n_data
    print('val_loss: {:.4f}\t val_acc: {:.4f}'.format(loss, acc))
    writer.add_scalar('val_loss', loss, step)
    writer.add_scalar('val_accuracy', acc, step)

In [12]:
if __name__ == '__main__':
    for step in range(args.epoch):
        print('step:{}'.format(step))
        train(step)
        test(step)

step:0
loss: 2.1338	 acc: 0.5052
val_loss: 5.7682	 val_acc: 0.1000
step:1
loss: 4.6641	 acc: 0.3021
val_loss: 7.1280	 val_acc: 0.2000
step:2
loss: 5.8580	 acc: 0.4688
val_loss: 11.1146	 val_acc: 0.2000
step:3
loss: 3.9602	 acc: 0.4896
val_loss: 10.7672	 val_acc: 0.2000
step:4
loss: 3.2895	 acc: 0.5885
val_loss: 7.0896	 val_acc: 0.1000
step:5
loss: 2.5448	 acc: 0.4948
val_loss: 4.2348	 val_acc: 0.3000
step:6
loss: 3.5615	 acc: 0.2812
val_loss: 4.5964	 val_acc: 0.2000
step:7
loss: 2.7282	 acc: 0.3385
val_loss: 3.0145	 val_acc: 0.5000
step:8
loss: 2.4555	 acc: 0.4688
val_loss: 3.6146	 val_acc: 0.5000
step:9
loss: 2.2177	 acc: 0.5729
val_loss: 2.0694	 val_acc: 0.2000
step:10
loss: 1.9041	 acc: 0.5833
val_loss: 2.1855	 val_acc: 0.6000
step:11
loss: 1.6485	 acc: 0.6406
val_loss: 3.9471	 val_acc: 0.4000
step:12
loss: 1.9141	 acc: 0.6146
val_loss: 3.0304	 val_acc: 0.5000
step:13
loss: 1.9377	 acc: 0.6458
val_loss: 4.4948	 val_acc: 0.2000
step:14
loss: 1.1959	 acc: 0.6719
val_loss: 3.5093	 val_